# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
#氣象
ETtoday_url = "https://www.ettoday.net/news/focus/%E7%94%9F%E6%B4%BB/%E6%B0%A3%E8%B1%A1/" 

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
# 打開瀏覽器連結到指定新聞類別的頁面
browser = webdriver.Chrome(executable_path='./Data/chromedriver')
browser.get(ETtoday_url)

In [4]:
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#

# 取得目前網頁的height
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    # 捲動到目前的最底
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print("更新中...")
    time.sleep(1.5)

    # 計算捲動完後目前的height
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    # 相等時代表到達最底，結束while
    if new_height == last_height:
        print("到達頁面最底部。")
        break
    
    # 將最新的height持續取代，直到碰到最底
    last_height = new_height

更新中...
更新中...
更新中...
更新中...
更新中...
更新中...
到達頁面最底部。


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
# 用來爬取外部連結內容
def getNewsDetailContent(link_url):
    response = requests.get(link_url)
    response.encoding = "utf-8"
    #print(resp.text)

    soup = BeautifulSoup(response.text, "html5lib")
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "html5lib")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[{}] {}\n".format(i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)

----------------------------------------------------------------------

[0] 2022年首道曙光在蘭嶼！　全台景點「迎日出」時刻表一次看

記者許維寧／台北報導
中央氣象局今天（12/27）公布「2021-2022年跨年日沒日出資料」，預測今年本島平地最晚日落時間位於台南市七股海堤。元旦第一道曙光則位於離島台東蘭嶼，本島則是在屏東恆春龍坑生態保護區。
氣象局公布2021-2022年跨年日沒日出資料，今年12月31日本島最晚日落時間為17時25分，地點位於台南七股海提，離島最晚日落時間則是17時29分，地點位於金門縣浯江溪建功嶼以及慈湖詞堤。
但氣象局提醒， 新北市淡水漁人碼頭、桃園市許厝港濕地、台中市高美濕地及金門、馬祖等部分地區受陸地天際線影響，難見日落於海平面。
至於民眾最期待的2022年第一道曙光，氣象局指出，第一道曙光時間為早上6時32分，地點位於蘭嶼東清；本島第一道曙光則是6時35分，地點在屏東縣龍坑生態保護區。
氣象屬也說明，新北市金山跳石海岸可見日出於基隆嶼；宜蘭縣外澳火車站以北附近可見日出於龜山島；臺東縣加路蘭遊憩區可見日出於綠島；大武濱海公園可見日出於蘭嶼。
▲跨年日出日沒資料。（圖／氣象局提供）※點圖可放大
----------------------------------------------------------------------

[1] 6山友秀「千元鈔」攻致敬　玉山白雪皚皚！鄭明典讚：雄偉壯麗

▲玉山氣象站員工忙著清掃太陽能板上的積雪。（圖／翻攝自鄭明典臉書）
生活中心／台北報導
中央氣象局持續發布低溫特報，玉山於今天（12/27）凌晨下起大雪，至今天上午9時左右已經有兩公分厚的積雪。中央氣象局局長鄭明典也表示，今天清晨玉山大雪，仍有山友登頂玉山北峰，六名山友也高舉千元大鈔的毛巾，與身後積雪的玉山進行合影。
玉山氣象站表示，今天清晨3時15分玉山開始下雪，至3時50分停雪。但5時20分左右又開始降雪，降雪持續至今天上午9時30分停止，目前玉山氣象站附近積雪已達2公分。
氣象局局長鄭明典也分享，今天一早仍有山友在風雪中前往玉山北峰頂的玉山氣象站，「厲害了！這種天氣，玉山北峰還有訪客上門！」玉山氣象站也隨即為山友燒炭火，一同取暖。
▲六名山友舉起千元大鈔

In [7]:
browser.close()